In [21]:
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json, GroupChat, GroupChatManager

# Load LLM inference endpoints from an env variable or a file
# See https://microsoft.github.io/autogen/docs/FAQ#set-your-api-endpoints
# and OAI_CONFIG_LIST_sample.json
config_list = config_list_from_json(env_or_file="OAI_CONFIG_LIST.json")
type(config_list)

list

In [8]:
assistant = AssistantAgent("assistant", llm_config={"config_list": config_list})


In [9]:
user_proxy = UserProxyAgent("user_proxy", code_execution_config={"work_dir": "coding", "use_docker": False}) # IMPORTANT: set to True to run code in docker, recommended


In [10]:
user_proxy.initiate_chat(assistant, message="Plot a chart of NVDA and TESLA stock price change YTD.")

user_proxy (to assistant):

Plot a chart of NVDA and TESLA stock price change YTD.

--------------------------------------------------------------------------------
assistant (to user_proxy):

To plot a chart of NVDA and TESLA stock price change year-to-date (YTD), we can use the 'yfinance' library in Python which allows us to download stock data from Yahoo Finance, and the 'matplotlib' library to plot the data. The following is the Python code.

# Python code
```python
import yfinance as yf
import matplotlib.pyplot as plt

# Retrieve NVDA and TESLA data from Yahoo Finance
nvda = yf.Ticker("NVDA")
tesla = yf.Ticker("TSLA")

# Download the YTD data
nvda_data = nvda.history(period="YTD")
tesla_data = tesla.history(period="YTD")

# Plot NVDA data
plt.plot(nvda_data.index, nvda_data['Close'], label='NVDA')

# Plot TESLA data
plt.plot(tesla_data.index, tesla_data['Close'], label='TESLA')

plt.xlabel('Date')
plt.ylabel('Close Price')
plt.title('NVDA and TESLA stock price change YTD')
plt.leg

KeyboardInterrupt: Interrupted by user

In [11]:
sales_mananger_prompt = """
You are Sales Manager(SM) at manufacturer details dealer.  Also there are Research Analyst (RA), Customer Service Reprisentative (CSR).  The business model of the company is very simple: Customers ask to buy parts, and a CSR (Customer Service Representative) looks at the request to see what parts the customer needs. Then, an RA (Research Analyst) finds the best prices for these parts online and tells the CSR. The CSR makes an offer and sends it to the customer. The customer can either buy the parts or have questions, and the CSR helps with these questions.

Description of the area of responsibility for each employee under your supervision.

Agent 1. Customer Service Reprisentative (CSR)
- classify customer request: extract details info from client email text body: brand name, part number, amount, model, serial number etc.
- answer on technical questions
- ping customer when it not responds
- generate commercial offer

Agent 2. Research Analyst (RA)
- search at CRM database: search detail by brand name or model or part number at company CRM database
- search at Google: search details at google with query

Your responsibility as a Sales Manager is to make decisions based on the information provided and to assign tasks to your subordinates. The task should be set in the following format:

Agent: responsible employee
State: this is one or more words in CamelCase that describe the current state
Task: a detailed description of the task in 1-2 sentences, which will be understandable to the employee.

[Example 1]
Agent: Customer Service Reprisentative (CSR)
State: PartClassification
Task: Customer sends email, you need to classify it and select information or generate response message.
"""

In [29]:
sales_manager_assistant = AssistantAgent(
    "sales_manager_assistant",
    sales_mananger_prompt,
    llm_config={"config_list": config_list})

In [30]:
csr_system_prompt = """
You are Customer Support Reprisentative at manufacturer details dealer.  To be concise your position would have abbreviature CSR. The business model of your company is quite simple:
- client, usually it company owner or reprisentative leave request on getting manufacturer details offer, this offer going to CRM
- CSR classify request, search details at CRM database and Google, get the best price
- if there is no info about detail or this information is outdated, CSR would communcate with detail manufacturer or supplier, it depends on which link CSR opened at Google with this detail.
- when info about detail was gathered CSR generate offer for client and continue to interact with custoemr. Price at offer was did with company commision.

So you're role at company is to communicate with clients and customers, answer on their common questions about delivery, dates, prices and on technical questions about details technical specifications from datasheets. You have quarterly KPI's and percent from each deal.

There are usual flow that you working every day, but consider that it could be different:
- classify customer request: extract details info from client email text body: brand name, part number, amount, model, serial number etc.
- answer on technical questions
- ping customer when it not responds
- generate commercial offer
"""

customer_support = AssistantAgent(
    name="customer_service_assistant",
    system_message= csr_system_prompt,
    llm_config={"config_list": config_list} 
)

In [31]:
user_proxy = UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    code_execution_config={
        "last_n_messages": 2,
        "work_dir": "groupchat",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    human_input_mode="TERMINATE",
)

In [32]:
groupchat = GroupChat(agents=[user_proxy, customer_support, sales_manager_assistant], messages=[], max_round=12)


In [33]:
manager = GroupChatManager(groupchat=groupchat, llm_config={"config_list": config_list})

In [ ]:
user_proxy.initiate_chat(
    manager, message="""
First email from customer: Ramees Khan | AREC <ramees@arecuae.com
Subject: Enquiry for Supply of Firetrol Battery Charger AS-2001 | AL RAMIZ
To: info@famaga.ae

Body:
Dear Sir/Ma’am,
We’re currently enquiring for Supply of Firetrol Battery Charger AS-2001 (Type 3AB LL-1580) Qty: 01 No. Please quote your best discounted prices and availability. I have attached a picture below for your reference

Thanking you.
Best Regards,
RAMEES KHAN
General Manager
M: +971-52-6927466 | E: ramees@arecuae.com
AL RAMIZ ELECT. CONT. LLC
A: Industrial Area 17, Sharjah - United Arab Emirates
A: P.O Box: 96286, Sharjah
T: +971-6-5356891
E: info@arecuae.com | W: www.arecuae.com
Transformers/Maintenance, Erection & Testing | Rewinding – Transformers, Motors & Generators | Cables – Laying, Termination, Testing – HT &
LT | Distribution Boards – MDB, SMDB, DB | Generator Alternator – Repair & Maintenance, Bus Bar Modification & Testing | MV, LV Electrical
Panel – Installation & Testing | Hiring of all types of Testing Instruments – Oil Filter Plants, Oil Tanks, Vacuum Pumps, Crimping Tools & Generators.
Disclaimer: This e-mail and any attachments may contain confidential and privileged information. If you are not the intended recipient, please
notify the sender immediately by return e-mail, delete this e-mail and destroy any copies. Any dissemination or use of this information by a
person other than the intended recipient is unauthorized and may be illegal
"""
)

In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

In [10]:
sales_mananger_prompt = """
You are Sales Manager(SM) at manufacturer details dealer. 
Also there are Research Analyst (RA), Customer Service Reprisentative (CSR).
The business model of the company is very simple: Customers ask to buy parts, and a CSR (Customer Service Representative) looks at the request to see what parts the customer needs. Then, an RA (Research Analyst) finds the best prices for these parts online and tells the CSR. The CSR makes an offer and sends it to the customer. The customer can either buy the parts or have questions, and the CSR helps with these questions.

Description of the area of responsibility for each employee under your supervision.

Agent 1. Customer Service Reprisentative (CSR)
- classify customer request: extract details info from client email text body: brand name, part number, amount, model, serial number etc.
- answer on technical questions
- ping customer when it not responds
- generate commercial offer

Agent 2. Research Analyst (RA)
- search at CRM database: search detail by brand name or model or part number at company CRM database
- search at Google: search details at google with query

Your responsibility as a Sales Manager is to make decisions based on the information provided and to assign tasks to your subordinates. 
The task should be set in the following format:

Starting below, you should follow this format:

Agent: responsible employee
State: this is one or more words in CamelCase that describe the current state
Task: a detailed description of the task in 1-2 sentences, which will be understandable to the employee.
Observation: the report from responsible employee about result completion of task
... (this Agent/State/Task/Observation can repeat N times)


[Example 1]
Agent: Customer Service Reprisentative (CSR)
State: PartClassification
Task: Customer sends email, you need to classify it and select information or generate response message.
Observation: I found the detail with brand name Firetrol and part number AS-2001

Agent: CSR
State: SearchAtCrmDb
Task: Try to search detail Firetrol AS-2001 at CRM database and exract all info
Observation: Detail was not found at CRM database.

Agent: CSR
State: SearchAtGoogle
Task: Try to search detail Firetrol AS-2001 and price at Google.
Observation: There are few links with price.
"""

csr_system_prompt = """
You are Customer Support Reprisentative at manufacturer details dealer.  To be concise your position would have abbreviature CSR. The business model of your company is quite simple:
- client, usually it company owner or reprisentative leave request on getting manufacturer details offer, this offer going to CRM
- CSR classify request, search details at CRM database and Google, get the best price
- if there is no info about detail or this information is outdated, CSR would communcate with detail manufacturer or supplier, it depends on which link CSR opened at Google with this detail.
- when info about detail was gathered CSR generate offer for client and continue to interact with custoemr. Price at offer was did with company commision.

So you're role at company is to communicate with clients and customers, answer on their common questions about delivery, dates, prices and on technical questions about details technical specifications from datasheets. You have quarterly KPI's and percent from each deal.

There are usual flow that you working every day, but consider that it could be different:
- classify customer request: extract details info from client email text body: brand name, part number, amount, model, serial number etc.
- answer on technical questions
- ping customer when it not responds
- generate commercial offer
"""


In [11]:
customer_request = """
First email from customer: Ramees Khan | AREC <ramees@arecuae.com
Subject: Enquiry for Supply of Firetrol Battery Charger AS-2001 | AL RAMIZ
To: info@famaga.ae

Body:
Dear Sir/Ma’am,
We’re currently enquiring for Supply of Firetrol Battery Charger AS-2001 (Type 3AB LL-1580) Qty: 01 No.
Please quote your best discounted prices and availability.
I have attached a picture below for your reference

Thanking you.
Best Regards,
RAMEES KHAN
General Manager
M: +971-52-6927466 | E: ramees@arecuae.com
AL RAMIZ ELECT. CONT. LLC
A: Industrial Area 17, Sharjah - United Arab Emirates
A: P.O Box: 96286, Sharjah
T: +971-6-5356891
E: info@arecuae.com | W: www.arecuae.com
Transformers/Maintenance, Erection & Testing | Rewinding – Transformers, Motors & Generators | Cables – Laying, Termination, Testing – HT &
LT | Distribution Boards – MDB, SMDB, DB | Generator Alternator – Repair & Maintenance, Bus Bar Modification & Testing | MV, LV Electrical
Panel – Installation & Testing | Hiring of all types of Testing Instruments – Oil Filter Plants, Oil Tanks, Vacuum Pumps, Crimping Tools & Generators.
Disclaimer: This e-mail and any attachments may contain confidential and privileged information. If you are not the intended recipient, please
notify the sender immediately by return e-mail, delete this e-mail and destroy any copies. Any dissemination or use of this information by a
person other than the intended recipient is unauthorized and may be illegal

"""

In [8]:
stop = ['\nObservation:', '\n\tObservation:']

In [25]:
class BaseRole:
    def __init__(self, content: str):
        self.content = content

class Assistant(BaseRole):
    pass

class Human(BaseRole):
    pass

In [27]:
Assistant("kek").content

'kek'

In [37]:
sales_manager_history = [
    Human(customer_request),
    Assistant("""
Agent: Customer Service Representative (CSR)
State: PartClassification
Task: We have received an email from a customer - Ramees Khan at AL RAMIZ ELECT. CONT. LLC. Please classify the information given in the email such as customer name, company name, mobile number, email id, and the part details for Firetrol Battery Charger AS-2001 (Type 3AB LL-1580) with Quantity: 01 No. Please also make a note of the customer's request for discounted prices and availability.
Observation: there were found fields Customer Name, Company Name, Mobile Number, Email ID, Brand Name, Part Number, Quantity
"""),
    Assistant("""
Agent: Research Analyst (RA)
State: SearchAtCrmDb
Task: Check in our CRM database for the Firetrol Battery Charger AS-2001 (Type 3AB LL-1580), please note down the details like pricing, availability and any discount applicable.
Observation: Detail was not found at our CRM
"""),
    Assistant("""
Agent: RA
State: SearchAtGoogle
Task: As we could not find the information about Firetrol Battery Charger AS-2001 (Type 3AB LL-1580) in our CRM, please try to find this detail on Google. Make note about prices, availability and any ongoing discounts on this product from various suppliers. 
Observation: There are few links at Google with price and few without price.
""")
]

In [39]:
def prepare_input(messages):
    final_text = ""
    for msg in messages:
        if isinstance(msg, Human):
            final_text += f'H: {msg.content}\n\n'
        elif isinstance(msg, Assistant):
            final_text += f'A: {msg.content}\n\n'
    return final_text
        

In [35]:
print(prepare_input())

H: 
First email from customer: Ramees Khan | AREC <ramees@arecuae.com
Subject: Enquiry for Supply of Firetrol Battery Charger AS-2001 | AL RAMIZ
To: info@famaga.ae

Body:
Dear Sir/Ma’am,
We’re currently enquiring for Supply of Firetrol Battery Charger AS-2001 (Type 3AB LL-1580) Qty: 01 No.
Please quote your best discounted prices and availability.
I have attached a picture below for your reference

Thanking you.
Best Regards,
RAMEES KHAN
General Manager
M: +971-52-6927466 | E: ramees@arecuae.com
AL RAMIZ ELECT. CONT. LLC
A: Industrial Area 17, Sharjah - United Arab Emirates
A: P.O Box: 96286, Sharjah
T: +971-6-5356891
E: info@arecuae.com | W: www.arecuae.com
Transformers/Maintenance, Erection & Testing | Rewinding – Transformers, Motors & Generators | Cables – Laying, Termination, Testing – HT &
LT | Distribution Boards – MDB, SMDB, DB | Generator Alternator – Repair & Maintenance, Bus Bar Modification & Testing | MV, LV Electrical
Panel – Installation & Testing | Hiring of all types o

In [38]:
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "assistant", "content": sales_mananger_prompt},
        {"role": "user", "content": prepare_input(sales_manager_history)}
    ],
    stop=stop,
    stream=False,
)
print(response.choices[0].message.content)

A: 
Agent: CSR
State: GenerateCommercialOffer
Task: Based on the information provided from Google, generate a detailed commercial offer for the Firetrol Battery Charger AS-2001 (Type 3AB LL-1580) with Quantity: 01 No. Include the price, availability, any available discounts, and a reference picture. Send this offer to the customer Ramees Khan at AL RAMIZ ELECT. CONT. LLC via email at ramees@arecuae.com. 


In [15]:
text = response.choices[0].message.content

In [16]:
import re

regex = r"Agent\s*\d*\s*:[\s]*(.*?)[\s]*State\s*\d*\s*:[\s]*(.*)[\s]*Task\s*\d*\s*:[\s]*(.*)"
action_match = re.search(regex, text, re.DOTALL)

In [19]:
agent = action_match.group(1).strip()
state = action_match.group(2).strip()
task = action_match.group(3).strip()

agent, state, task

('Customer Service Representative (CSR)',
 'PartClassification',
 "We have received an email from a customer - Ramees Khan at AL RAMIZ ELECT. CONT. LLC. Please classify the information given in the email such as customer name, company name, mobile number, email id, and the part details for Firetrol Battery Charger AS-2001 (Type 3AB LL-1580) with Quantity: 01 No. Please also make a note of the customer's request for discounted prices and availability.")

In [45]:
first_csr_request = """
We have received an email from a customer - Ramees Khan at AL RAMIZ ELECT. CONT. LLC. Please classify the information given in the email such as customer name, company name, mobile number, email id, and the part details for Firetrol Battery Charger AS-2001 (Type 3AB LL-1580) with Quantity: 01 No. Please also make a note of the customer's request for discounted prices and availability.

Subject: Enquiry for Supply of Firetrol Battery Charger AS-2001 | AL RAMIZ
To: info@famaga.ae

Body:
Dear Sir/Ma’am,
We’re currently enquiring for Supply of Firetrol Battery Charger AS-2001 (Type 3AB LL-1580) Qty: 01 No.
Please quote your best discounted prices and availability.
I have attached a picture below for your reference

Thanking you.
Best Regards,
RAMEES KHAN
General Manager
M: +971-52-6927466 | E: ramees@arecuae.com
AL RAMIZ ELECT. CONT. LLC
A: Industrial Area 17, Sharjah - United Arab Emirates
A: P.O Box: 96286, Sharjah
T: +971-6-5356891
E: info@arecuae.com | W: www.arecuae.com
Transformers/Maintenance, Erection & Testing | Rewinding – Transformers, Motors & Generators | Cables – Laying, Termination, Testing – HT &
LT | Distribution Boards – MDB, SMDB, DB | Generator Alternator – Repair & Maintenance, Bus Bar Modification & Testing | MV, LV Electrical
Panel – Installation & Testing | Hiring of all types of Testing Instruments – Oil Filter Plants, Oil Tanks, Vacuum Pumps, Crimping Tools & Generators.
Disclaimer: This e-mail and any attachments may contain confidential and privileged information. If you are not the intended recipient, please
notify the sender immediately by return e-mail, delete this e-mail and destroy any copies. Any dissemination or use of this information by a
person other than the intended recipient is unauthorized and may be illegal
"""


In [46]:
csr_history = [
    Human(first_csr_request),
    Assistant("""
Customer Name: Ramees Khan
Company Name: AL RAMIZ ELECT. CONT. LLC
Mobile Number: +971-52-6927466
Email ID: ramees@arecuae.com

Part Details: 
Brand Name: Firetrol
Part Number: AS-2001 (Type 3AB LL-1580)
Quantity: 01 
"""),
    Human("Check in our CRM database for the Firetrol Battery Charger AS-2001 (Type 3AB LL-1580), please note down the details like pricing, availability and any discount applicable."),
    Assistant("""
Here info about Firetrol AS-2001 at CRM database:

Id: 746602,
Brand name: Firetrol Protection Systems,
Part number: AS-2001,
Description: Battery Charge
"""),
    Human("As we could not find the information about Firetrol Battery Charger AS-2001 (Type 3AB LL-1580) in our CRM, please try to find this detail on Google. Make note about prices, availability and any ongoing discounts on this product from various suppliers."),
    Assistant("""
Here is price info of Firetrol AS-2001 at google:

Here are the notes with prices in the requested format:

Title: Firetrol AS-2001 Battery Charger, Untested. Looks Great, Will Guarantee
Price: $900.00
Link: picclick.com

Title: Firetrol AS-2001 Battery Charger (aka LL-1580) - WS Supply Store
Price: $1170.00
Link: wssupplystore.com

Title: Firetrol AS-2001 Battery Charger - Board Repair
Price: $1699.20
Link: board-repair.com
    
""")
    
]

In [49]:
task = "Based on the information provided from Google, generate a detailed commercial offer for the Firetrol Battery Charger AS-2001 (Type 3AB LL-1580) with Quantity: 01 No. Include the price, availability, any available discounts, and a reference picture. Send this offer to the customer Ramees Khan at AL RAMIZ ELECT. CONT. LLC via email at ramees@arecuae.com."

response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "assistant", "content": csr_system_prompt},
        {"role": "user", "content": f"""
{prepare_input(csr_history)}

H: {task}
"""}
    ],
    stop=stop,
    stream=False,
)
print(response.choices[0].message.content)

A: 
Dear Ramees Khan,

I hope my email finds you well. Thank you for your interest in our products.

As per your request for Firetrol Battery Charger AS-2001 (Type 3AB LL-1580), please find the detailed commercial offer below:

Product information:
Brand: Firetrol
Model: AS-2001 (Type 3AB LL-1580)

Quantity: 01 No.

Availability: Since Firetrol Battery Charger AS-2001 (Type 3AB LL-1580) is limited in stock on the market, we would strongly recommend placing an order soon to secure one.

Pricing:
Based on the market research we have conducted, prices for the Firetrol Battery Charger AS-2001 (Type 3AB LL-1580) range between $900.00 and $1699.20. However, we are pleased to offer you a competitive price of $xxxx.xx (To be defined based on commission and discount applied).

Please find the reference picture attached.

Should you require any further information or have any queries, please do not hesitate to contact us.

Looking forward to your positive response.

Best Regards,

[Your Name]
Cu

In [43]:
print(prepare_input(csr_history))

H: We have received an email from a customer - Ramees Khan at AL RAMIZ ELECT. CONT. LLC. Please classify the information given in the email such as customer name, company name, mobile number, email id, and the part details for Firetrol Battery Charger AS-2001 (Type 3AB LL-1580) with Quantity: 01 No. Please also make a note of the customer's request for discounted prices and availability.

A: 
Customer Name: Ramees Khan
Company Name: AL RAMIZ ELECT. CONT. LLC
Mobile Number: +971-52-6927466
Email ID: ramees@arecuae.com

Part Details: 
Brand Name: Firetrol
Part Number: AS-2001 (Type 3AB LL-1580)
Quantity: 01 


H: Check in our CRM database for the Firetrol Battery Charger AS-2001 (Type 3AB LL-1580), please note down the details like pricing, availability and any discount applicable.

A: 
Here info about Firetrol AS-2001 at CRM database:

Id: 746602,
Brand name: Firetrol Protection Systems,
Part number: AS-2001,
Description: Battery Charge


H: As we could not find the information about Fi

In [53]:
import psycopg2
from psycopg2 import OperationalError

def create_connection(db_name, db_user, db_password, db_host, db_port):
    connection = None
    try:
        connection = psycopg2.connect(
            database=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port,
            sslmode='disable'
        )
        print("Connection to PostgreSQL DB successful")
    except OperationalError as e:
        print(f"The error '{e}' occurred")
    return connection


In [54]:
db_name = "famaga"
db_user = "admin"
db_password = "5tgb%TGB"
db_host = "famaga-db.neon-dev.us"
db_port = "443"


connection = create_connection(db_name, db_user, db_password, db_host, db_port)

The error 'connection to server at "famaga-db.neon-dev.us" (172.67.180.128), port 443 failed: expected authentication request from server, but received H
' occurred


In [57]:
db_name = "famaga"
db_user = "admin"
db_password = "5tgb%TGB"
db_host = "154.38.160.240"
db_port = "45043"


connection = create_connection(db_name, db_user, db_password, db_host, db_port)

Connection to PostgreSQL DB successful
